* Normalize the primary keys
    * School Primary Keys
    * District Primary Keys
* Reindexing columns
* Bad value
* Transform columns dtypes
* Extracting Pupil-Teature Ratio
* Build feature from dsClassSize
* Drop non-numeric columns
* Joining dfs
* Replace % by and convert to float
* Convert categorical columns to dummies
* Useful categorical can't be drop
* Generate train.csv and target.csv


In [2]:
import pandas as pd
# Load the data
dsProgReports = pd.read_csv('F:/data_ware/NYC_Schools/School_Progress_Reports_-_All_Schools_-_2009-10.csv')
dsDistrict = pd.read_csv('F:/data_ware/NYC_Schools/School_District_Breakdowns.csv')
dsClassSize = pd.read_csv('F:/data_ware/NYC_Schools/2009-10_Class_Size_-_School-level_Detail.csv')
dsAttendEnroll = pd.read_csv('F:/data_ware/NYC_Schools/School_Attendance_and_Enrollment_Statistics_by_District__2010-11_.csv')[:-2] #last two rows are bad
dsSATs = pd.read_csv('F:/data_ware/NYC_Schools/SAT__College_Board__2010_School_Level_Results.csv') # Dependent

In [3]:
dsProgReports.columns

Index(['DBN', 'DISTRICT', 'SCHOOL', 'PRINCIPAL', 'PROGRESS REPORT TYPE',
       'SCHOOL LEVEL*', 'PEER INDEX*', '2009-2010 OVERALL GRADE',
       '2009-2010 OVERALL SCORE', '2009-2010 ENVIRONMENT CATEGORY SCORE',
       '2009-2010 ENVIRONMENT GRADE', '2009-2010 PERFORMANCE CATEGORY SCORE',
       '2009-2010 PERFORMANCE GRADE', '2009-2010 PROGRESS CATEGORY SCORE',
       '2009-2010 PROGRESS GRADE', '2009-2010 ADDITIONAL CREDIT',
       '2008-09 PROGRESS REPORT GRADE'],
      dtype='object')

In [99]:
dsSATs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 460 entries, 0 to 459
Data columns (total 6 columns):
DBN                      460 non-null object
School Name              460 non-null object
Number of Test Takers    460 non-null object
Critical Reading Mean    460 non-null object
Mathematics Mean         460 non-null object
Writing Mean             460 non-null object
dtypes: object(6)
memory usage: 25.2+ KB


#Primary Keys-schools
Before we can join these three datasets together, we need to normalize their primary keys. Below we see the mismatch between the way the DBN (school id) field is represented in the different datasets. We then write code to normalize the keys and correct this problem.

In [100]:
pd.DataFrame(data=[dsProgReports['DBN'].head(10), dsSATs['DBN'].head(10), dsClassSize['SCHOOL CODE'].head(10)])

,0,1,2,3,4,5,6,7,8,9
DBN,01M015,01M019,01M020,01M034,01M063,01M064,01M110,01M134,01M137,01M140
DBN,01M292,01M448,01M450,01M458,01M509,01M515,01M539,01M650,01M696,02M047
SCHOOL CODE,M015,M015,M015,M015,M015,M015,M015,M015,M015,M015


??take(range(4)), show(), take(4)

In [101]:
#Strip the first two characters off the DBNs so we can join to School Code
dsProgReports.DBN = dsProgReports.DBN.map(lambda x: x[2:])
dsSATs.DBN = dsSATs.DBN.map(lambda x: x[2:])

#We can now see the keys match
pd.DataFrame(data=[dsProgReports['DBN'].head(5), dsSATs['DBN'].head(5), dsClassSize['SCHOOL CODE'].head(5)])
#dsProgReports['DBN'].head(10)

,0,1,2,3,4
DBN,M015,M019,M020,M034,M063
DBN,M292,M448,M450,M458,M509
SCHOOL CODE,M015,M015,M015,M015,M015


#Primary Keys-District
We have a similar story with the district foreign keys. Again, we need to normalize the keys. The only additional complexity here is that dsProgReports['DISTRICT'] is typed numerically, whereas the other two district keys are typed as string. We do some type conversions following the key munging.


In [102]:
#Show the key mismatchs
#For variety's sake, using slicing ([:3]) syntax instead of .take()
pd.DataFrame(data=[dsProgReports['DISTRICT'][:3], 
                   dsDistrict['JURISDICTION NAME'][:3], 
                   dsAttendEnroll['District'][:3]])

,0,1,2
DISTRICT,1,1,1
JURISDICTION NAME,CSD 01 Manhattan,CSD 02 Manhattan,CSD 03 Manhattan
District,DISTRICT 01,DISTRICT 02,DISTRICT 03


In [103]:
#Extract well-formed district key values
#Note the astype(int) at the end of these lines to coerce the column to a numeric type
import re
dsDistrict['JURISDICTION NAME'] = dsDistrict['JURISDICTION NAME'].map(lambda x: re.match( r'([A-Za-z]*\s)([0-9]*)', x).group(2)).astype(int)
dsAttendEnroll.District = dsAttendEnroll.District.map(lambda x: x[-2:]).astype(int)

#We can now see the keys match
pd.DataFrame(data=[dsProgReports['DISTRICT'][:3], dsDistrict['JURISDICTION NAME'][:3], dsAttendEnroll['District'][:3]])

,0,1,2
DISTRICT,1,1,1
JURISDICTION NAME,1,2,3
District,1,2,3


#Reindexing the columns

In [104]:
#Reindexing
dsProgReports = dsProgReports.set_index('DBN')
dsDistrict = dsDistrict.set_index('JURISDICTION NAME')
dsClassSize = dsClassSize.set_index('SCHOOL CODE')
dsAttendEnroll = dsAttendEnroll.set_index('District')
dsSATs = dsSATs.set_index('DBN')

# Bad value

In [105]:
#We can see the bad value
dsSATs['Critical Reading Mean'].head(5)

DBN
M292    391
M448    394
M450    418
M458    385
M509      s
Name: Critical Reading Mean, dtype: object

In [106]:
#Now we filter it out

#We create a boolean vector mask. Open question as to whether this semantically ideal...
mask = dsSATs['Number of Test Takers'].map(lambda x: x != 's')
dsSATs = dsSATs[mask]
#We can see those values are gone
dsSATs['Critical Reading Mean'].head(5)

DBN
M292    391
M448    394
M450    418
M458    385
M515    314
Name: Critical Reading Mean, dtype: object

# Tranform columns dtypes

In [107]:
dsSATs.dtypes

School Name              object
Number of Test Takers    object
Critical Reading Mean    object
Mathematics Mean         object
Writing Mean             object
dtype: object

In [108]:
#Cast fields to integers. Ideally we should not need to be this explicit.
dsSATs['Number of Test Takers'] = dsSATs['Number of Test Takers'].astype(int)
dsSATs['Critical Reading Mean'] = dsSATs['Critical Reading Mean'].astype(int)
dsSATs['Mathematics Mean'] = dsSATs['Mathematics Mean'].astype(int)
dsSATs['Writing Mean'] = dsSATs['Writing Mean'].astype(int)
dsSATs.dtypes

School Name              object
Number of Test Takers     int32
Critical Reading Mean     int32
Mathematics Mean          int32
Writing Mean              int32
dtype: object

#Extracting the Pupil-Teacher Ratio

In [109]:
dsClassSize.tail(5)

,BORO,CSD,SCHOOL NAME,GRADE,PROGRAM TYPE,CORE SUBJECT (MS CORE and 9-12 ONLY),CORE COURSE (MS CORE and 9-12 ONLY),SERVICE CATEGORY(K-9* ONLY),NUMBER OF CLASSES,TOTAL REGISTER,AVERAGE CLASS SIZE,SIZE OF SMALLEST CLASS,SIZE OF LARGEST CLASS,DATA SOURCE,SCHOOLWIDE PUPIL-TEACHER RATIO
SCHOOL CODE,,,,,,,,,,,,,,,
K564,K,32,BUSHWICK COMMUNITY HIGH SCHOOL,09-12,GEN ED,SOCIAL STUDIES,ECONOMICS,-,1,26,26.0,26,26,STARS,NaN
K564,K,32,BUSHWICK COMMUNITY HIGH SCHOOL,09-12,GEN ED,SOCIAL STUDIES,OTHER,-,14,343,24.5,16,30,STARS,NaN
K564,K,32,BUSHWICK COMMUNITY HIGH SCHOOL,09-12,CTT,SOCIAL STUDIES,ECONOMICS,-,1,25,25.0,25,25,STARS,NaN
K564,K,32,BUSHWICK COMMUNITY HIGH SCHOOL,09-12,CTT,SOCIAL STUDIES,OTHER,-,2,54,27.0,23,31,STARS,NaN
K564,K,32,BUSHWICK COMMUNITY HIGH SCHOOL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.4


In [110]:
dsClassSize.columns, dsClassSize.take([0,1,10]).values

(Index(['BORO', 'CSD', 'SCHOOL NAME', 'GRADE ', 'PROGRAM TYPE',
        'CORE SUBJECT (MS CORE and 9-12 ONLY)',
        'CORE COURSE (MS CORE and 9-12 ONLY)', 'SERVICE CATEGORY(K-9* ONLY)',
        'NUMBER OF CLASSES', 'TOTAL REGISTER', 'AVERAGE CLASS SIZE',
        'SIZE OF SMALLEST CLASS', 'SIZE OF LARGEST CLASS', 'DATA SOURCE',
        'SCHOOLWIDE PUPIL-TEACHER RATIO'],
       dtype='object'),
 array([['M', 1, 'P.S. 015 ROBERTO CLEMENTE', '0K', 'GEN ED', '-', '-', '-',
         1.0, 21.0, 21.0, 21.0, 21.0, 'ATS', nan],
        ['M', 1, 'P.S. 015 ROBERTO CLEMENTE', '0K', 'CTT', '-', '-', '-',
         1.0, 21.0, 21.0, 21.0, 21.0, 'ATS', nan],
        ['M', 1, 'P.S. 015 ROBERTO CLEMENTE', nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, 8.9]], dtype=object))

In [111]:
#Extracting the Pupil-Teacher Ratio

#Take the column
dsPupilTeacher = dsClassSize.filter(['SCHOOLWIDE PUPIL-TEACHER RATIO'])
#And filter out blank rows
mask = dsPupilTeacher['SCHOOLWIDE PUPIL-TEACHER RATIO'].map(lambda x: x > 0)
dsPupilTeacher = dsPupilTeacher[mask]
#Then drop from the original dataset
dsClassSize = dsClassSize.drop('SCHOOLWIDE PUPIL-TEACHER RATIO', axis=1)

#Drop non-numeric fields

In [112]:
#Drop non-numeric fields
dsClassSize = dsClassSize.drop(['BORO','CSD','SCHOOL NAME','GRADE ','PROGRAM TYPE',\
'CORE SUBJECT (MS CORE and 9-12 ONLY)','CORE COURSE (MS CORE and 9-12 ONLY)',\
'SERVICE CATEGORY(K-9* ONLY)','DATA SOURCE'], axis=1)

#Build features from dsClassSize

In [117]:
import numpy as np
#Build features from dsClassSize
#In this case, we'll take the max, min, and mean
#Semantically equivalent to select min(*), max(*), mean(*) from dsClassSize group by SCHOOL NAME
#Note that SCHOOL NAME is not referenced explicitly below because it is the index of the dataframe
grouped = dsClassSize.groupby(level=0)
dsClassSize = grouped.aggregate(np.max).\
    join(grouped.aggregate(np.min), lsuffix=".max").\
    join(grouped.aggregate(np.mean), lsuffix=".min", rsuffix=".mean").\
    join(dsPupilTeacher)

print (dsClassSize.columns)
dsClassSize.head(5)

Index(['NUMBER OF CLASSES.max', 'TOTAL REGISTER.max', 'AVERAGE CLASS SIZE.max',
       'SIZE OF SMALLEST CLASS.max', 'SIZE OF LARGEST CLASS.max',
       'NUMBER OF CLASSES.min', 'TOTAL REGISTER.min', 'AVERAGE CLASS SIZE.min',
       'SIZE OF SMALLEST CLASS.min', 'SIZE OF LARGEST CLASS.min',
       'NUMBER OF CLASSES.mean', 'TOTAL REGISTER.mean',
       'AVERAGE CLASS SIZE.mean', 'SIZE OF SMALLEST CLASS.mean',
       'SIZE OF LARGEST CLASS.mean', 'SCHOOLWIDE PUPIL-TEACHER RATIO'],
      dtype='object')


,NUMBER OF CLASSES.max,TOTAL REGISTER.max,AVERAGE CLASS SIZE.max,SIZE OF SMALLEST CLASS.max,SIZE OF LARGEST CLASS.max,NUMBER OF CLASSES.min,TOTAL REGISTER.min,AVERAGE CLASS SIZE.min,SIZE OF SMALLEST CLASS.min,SIZE OF LARGEST CLASS.min,NUMBER OF CLASSES.mean,TOTAL REGISTER.mean,AVERAGE CLASS SIZE.mean,SIZE OF SMALLEST CLASS.mean,SIZE OF LARGEST CLASS.mean,SCHOOLWIDE PUPIL-TEACHER RATIO
SCHOOL CODE,,,,,,,,,,,,,,,,
K001,7,158,27.3,25,31,1,12,12.0,12,12,2.611111,56.277778,20.822222,19.944444,21.555556,13.3
K002,11,293,27.7,26,31,1,10,8.0,2,10,3.333333,68.750000,19.316667,16.166667,21.083333,10.7
K003,4,76,28.0,28,28,1,12,9.5,8,11,1.933333,39.800000,20.646667,20.333333,20.866667,14.7
K005,4,75,27.0,27,27,1,21,9.3,8,12,2.875000,54.750000,19.612500,17.625000,21.125000,14.6
K006,4,125,31.3,31,32,1,11,10.7,8,11,2.545455,60.000000,21.927273,20.727273,23.181818,14.6


In [118]:
mask = dsProgReports['SCHOOL LEVEL*'].map(lambda x: x == 'High School')
dsProgReports = dsProgReports[mask]

#Joining the dfs

In [119]:
final = dsSATs.join(dsClassSize).\
join(dsProgReports).\
merge(dsDistrict, left_on='DISTRICT', right_index=True).\
merge(dsAttendEnroll, left_on='DISTRICT', right_index=True)

In [125]:
final.shape
final.columns

Index(['School Name', 'Number of Test Takers', 'Critical Reading Mean',
       'Mathematics Mean', 'Writing Mean', 'NUMBER OF CLASSES.max',
       'TOTAL REGISTER.max', 'AVERAGE CLASS SIZE.max',
       'SIZE OF SMALLEST CLASS.max', 'SIZE OF LARGEST CLASS.max',
       'NUMBER OF CLASSES.min', 'TOTAL REGISTER.min', 'AVERAGE CLASS SIZE.min',
       'SIZE OF SMALLEST CLASS.min', 'SIZE OF LARGEST CLASS.min',
       'NUMBER OF CLASSES.mean', 'TOTAL REGISTER.mean',
       'AVERAGE CLASS SIZE.mean', 'SIZE OF SMALLEST CLASS.mean',
       'SIZE OF LARGEST CLASS.mean', 'SCHOOLWIDE PUPIL-TEACHER RATIO',
       'DISTRICT', 'SCHOOL', 'PRINCIPAL', 'PROGRESS REPORT TYPE',
       'SCHOOL LEVEL*', 'PEER INDEX*', '2009-2010 OVERALL GRADE',
       '2009-2010 OVERALL SCORE', '2009-2010 ENVIRONMENT CATEGORY SCORE',
       '2009-2010 ENVIRONMENT GRADE', '2009-2010 PERFORMANCE CATEGORY SCORE',
       '2009-2010 PERFORMANCE GRADE', '2009-2010 PROGRESS CATEGORY SCORE',
       '2009-2010 PROGRESS GRADE', '2009-2

#Drop string columns

In [127]:
final.dtypes[final.dtypes.map(lambda x: x=='object')]

School Name                      object
SCHOOL                           object
PRINCIPAL                        object
PROGRESS REPORT TYPE             object
SCHOOL LEVEL*                    object
2009-2010 OVERALL GRADE          object
2009-2010 ENVIRONMENT GRADE      object
2009-2010 PERFORMANCE GRADE      object
2009-2010 PROGRESS GRADE         object
2008-09 PROGRESS REPORT GRADE    object
YTD % Attendance (Avg)           object
dtype: object

In [128]:
#Just drop string columns.
#In theory we could build features out of some of these, but it is impractical here
final = final.drop(['School Name','SCHOOL','PRINCIPAL','SCHOOL LEVEL*','PROGRESS REPORT TYPE'],axis=1)

#Remove % signs and convert to float

In [130]:
final['YTD % Attendance (Avg)'].head(5)

DBN
M292    91.18%
M448    91.18%
M450    91.18%
M539    91.18%
M696    91.18%
Name: YTD % Attendance (Avg), dtype: object

In [131]:
#Remove % signs and convert to float
final['YTD % Attendance (Avg)'] = final['YTD % Attendance (Avg)'].map(lambda x: x.replace("%","")).astype(float)

#Usefull categorical varible can't be drop
We can see above that the remaining non-numeric field are grades . Intuitively, they might be important so we don't want to drop them, but in order to get a pure feature matrix we need numeric values. The approach we'll use here is to explode these into multiple boolean columns. Some machine learning libraries effectively do this for you under the covers, but when the cardinality of the categorical variable is relatively low, it's nice to be explicit about it.

In [1]:
final.dtypes[final.dtypes.map(lambda x: x == 'object')]

NameError: name 'final' is not defined

In [138]:
final['2009-2010 ENVIRONMENT GRADE'].head(5)

DBN
M292    F
M448    F
M450    A
M539    C
M696    A
Name: 2009-2010 ENVIRONMENT GRADE, dtype: object

In [142]:
final['2009-2010 ENVIRONMENT GRADE'].value_counts()

B    88
A    86
C    79
D    43
F    27
dtype: int64

In [150]:
final.select_dtypes(include=['object']).head(10)

,2009-2010 OVERALL GRADE,2009-2010 ENVIRONMENT GRADE,2009-2010 PERFORMANCE GRADE,2009-2010 PROGRESS GRADE,2008-09 PROGRESS REPORT GRADE
DBN,,,,,
M292,D,F,C,F,B
M448,C,F,D,C,D
M450,A,A,A,B,A
M539,B,C,A,B,A
M696,B,A,B,C,B
M047,A,A,B,A,A
M288,A,B,A,A,A
M294,A,A,A,A,A
M296,A,B,A,A,A


In [156]:
gradeCols = ['2009-2010 OVERALL GRADE','2009-2010 ENVIRONMENT GRADE','2009-2010 PERFORMANCE GRADE',
             '2009-2010 PROGRESS GRADE','2008-09 PROGRESS REPORT GRADE']
grades = np.unique(final[gradeCols].dropna().values) #[nan, A, B, C, D, F]
grades

array(['A', 'B', 'C', 'D', 'F'], dtype=object)

#Convert categorical varible to multiple boolean columns(dummies)


In [159]:
pd.get_dummies(final).columns

Index(['Number of Test Takers', 'Critical Reading Mean', 'Mathematics Mean',
       'Writing Mean', 'NUMBER OF CLASSES.max', 'TOTAL REGISTER.max',
       'AVERAGE CLASS SIZE.max', 'SIZE OF SMALLEST CLASS.max',
       'SIZE OF LARGEST CLASS.max', 'NUMBER OF CLASSES.min',
       'TOTAL REGISTER.min', 'AVERAGE CLASS SIZE.min',
       'SIZE OF SMALLEST CLASS.min', 'SIZE OF LARGEST CLASS.min',
       'NUMBER OF CLASSES.mean', 'TOTAL REGISTER.mean',
       'AVERAGE CLASS SIZE.mean', 'SIZE OF SMALLEST CLASS.mean',
       'SIZE OF LARGEST CLASS.mean', 'SCHOOLWIDE PUPIL-TEACHER RATIO',
       'DISTRICT', 'PEER INDEX*', '2009-2010 OVERALL SCORE',
       '2009-2010 ENVIRONMENT CATEGORY SCORE',
       '2009-2010 PERFORMANCE CATEGORY SCORE',
       '2009-2010 PROGRESS CATEGORY SCORE', '2009-2010 ADDITIONAL CREDIT',
       'COUNT PARTICIPANTS', 'COUNT FEMALE', 'PERCENT FEMALE', 'COUNT MALE',
       'PERCENT MALE', 'COUNT GENDER UNKNOWN', 'PERCENT GENDER UNKNOWN',
       'COUNT GENDER TOTAL', 'PERCEN

#Generate train.csv and target.csv

In [160]:
#Uncomment to generate csv files 
final.drop(['Critical Reading Mean','Mathematics Mean','Writing Mean'],axis=1).to_csv('F:/data_ware/NYC_Schools/train.csv') 
final.filter(['Critical Reading Mean','Mathematics Mean','Writing Mean']).to_csv('F:/data_ware/NYC_Schools/target.csv')